# Homework 1 Week 1

## Part 1: Introduction to PyTorch

### Concept Questions

Answer the following questions in complete sentences, providing as much detail as neccessary to thoroughly answer.

* What is the PyTorch equivalent of:

    * `tf.keras.Model`

    * `model.compile()`

    * `model.fit()`

* In Keras, where is the forward pass defined? In PyTorch, where is it defined?

* Why does PyTorch require calling `loss. backward()` explicitly?

* What does `optimizer.step()` do, conceptually?

### Create Neural Networks in Python
For this assignment you are to create a convolutional neural network to classify the Fashion MNIST data set.
* Import the data set from keras, similar to how the MNSIT data set is imported in lecture notes.
* Plan a CNN that could be used to classify the data. You can use the one from the lecture notes, your solution from DSC 340, or create one from scratch. Note that for this assignment accuracy does not matter. Implement this neural network using both formats shown in the lecture notes (with and without `nn.Sequential`).
* Make the following changes to one of your base neural network. Make sure that all changes are shown, either in one network or across several networks:
    * Add at least one more layer, of any type.
    * Change the activation function of at least one layer.
    * Change the learning rate of the optimizer.
    * Add dropout between any layers of the network.
    * Add a batch size to your training loop. In the examples shown in the lecture notes, all of the data is passed at once.
    * Add early stopping to the training process.